In [24]:
import numpy as np
import pandas as pd
from collections import Counter
from Data import load_data

rng = np.random.default_rng(42)

players, points, rebounds, assists, weights = load_data("./nbasalariespoints.csv", "Player", "PS/G▼", "TRB", "AST", "SALARY")

In [25]:
# Create a population of num_teams x team_size, with random values from 0-last index
# Each individual represents a team, and each team has 15 players
# **has uniform distribution**
def initializePopulation(pop_size, team_size, max_index):
    return rng.integers(0, max_index, (pop_size, team_size), dtype=int)

def calculateValue(individual, obj):
    return np.sum(obj[individual])

def calculateWeight(individual, weights):
    return np.sum(weights[individual])


# Change so that instead of objectives being constrained, it calculates the distance between the team's stats
# and an idea point that the user inputs. Then use 1/dist as the fitness.
# Individuals that are far away from the point will have low fitness, and ones that are close
# will have high fitness. Then, use the best as parents for the next population.
# Include a step of calculating how many times a certian player appears in the top (10? 15?) teams
# and add them into the distrubution that many times
# This leads to a non-uniform distribution, which allows for better players to be picked more often

# Sets the fitness to -inf if the weight exceeds the knapsack capacity, or if there are duplicate players (can't draft the same player twice)
# Otherwise, sets the fitness to the value of the team
# Since there is a limit on how many players can be in each team, don't use value to weight ratio as the fitness
def calculateFitness(individual, weights, capacity, ideal_point, objectives):
    val_list = []
    total = 0

    for objective in objectives:
        val_list.append(calculateValue(individual, objective))
    
    for i in range(len(ideal_point)):
        total += (ideal_point[i] - val_list[i]) ** 2
    dist = np.sqrt(total)

    # Use 1/dist as fitness
    fitness = 1/dist

    # Can't choose the same player twice
    if (len(set(individual)) < 15) or (calculateWeight(individual, weights) > capacity):
        fitness = 0
    return fitness

# Selects the two best as the parents
def adjustDistribution(capacity, weights, population, ideal_point, objectives):
    fitness = []
    for individual in population:
        fitness.append(calculateFitness(individual, weights, capacity, ideal_point, objectives))
    fitness = np.array(fitness).sort()[::-1]

    # Calculate how many times each player from the fit population apears
    # then add that many copies into the distribution before sampling again
    # This (effectively) leads to weighting each player, allowing for the better players to be chosen more often 
    # Use counter from collections 
    counter = Counter()
    for i in range(15):
        counter += Counter(fitness[i])
    print(counter)

# Generates a population of the same size as the initial population
# Selects each gene randomly from one of the parents
def generatePopulation(parents, pop_size, team_size, length, mutation_rate=.1):
    population = np.zeros((pop_size, team_size), dtype=int)
    population[0], population[1], population[2] = parents

    for i in range(3, pop_size):
        for j in range(team_size):
            parent_choice = rng.integers(0, 3)
            population[i][j] = parents[parent_choice][j]
            
            # Mutation
            if rng.random() < mutation_rate:
                population[i][j] = rng.integers(0, length)
    return population

# Creates an initial population
# Saves the mean value for the population in each training iteration to plot
# Finds to two best as parents
# Creates a new population using the parents
# Saves only the valid solutions in the final population and returns the one with the highest value
def knapSackGenetic(capacity, weights, training_length, pop_size, team_size, ideal_point, objectives):
    max_index = len(weights) - 1
    population = initializePopulation(pop_size * (len(objectives) + 40), team_size, max_index)

    for i in range(training_length):
        parents = adjustDistribution(capacity, weights, population, ideal_point, objectives)
        population = generatePopulation(parents, pop_size, team_size, max_index)
    
    valid = []
    for individual in population:
        if calculateFitness(individual, weights, capacity, epsilon, primary, objectives) == -np.inf: continue
        valid.append(individual)

    valid = np.array(valid)
    print(f"Length of valid: {len(valid)}")
    if len(valid) == 0:

        return -1   
    indices = np.argsort([val for val in [calculateValue(ind, primary) for ind in valid]])
    return valid[indices[-1]]


In [26]:
ideal_point = [250, 20]
capacity = 150_000_000
training_length = 1000
pop_size = 500
team_size = 15
individual = knapSackGenetic(capacity, weights, training_length, pop_size, team_size, ideal_point, [points, rebounds, assists])


TypeError: 'NoneType' object is not subscriptable